In [69]:
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, classification_report
from sklearn.preprocessing import OrdinalEncoder
import lightgbm


In [70]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
#Original = pd.read_csv("diabetes_dataset.csv")

In [71]:
df_train

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,...,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,...,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,...,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,...,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,...,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699995,699995,29,1,59,6.9,5.2,1.5,26.1,0.88,133,...,Female,Hispanic,Postgraduate,Upper-Middle,Former,Employed,0,0,0,0.0
699996,699996,46,2,72,7.7,7.7,3.8,25.5,0.85,106,...,Female,Hispanic,Graduate,Upper-Middle,Former,Employed,0,0,1,1.0
699997,699997,35,1,50,5.6,6.1,6.4,26.9,0.88,127,...,Female,White,Graduate,Middle,Never,Employed,0,0,0,1.0
699998,699998,49,2,70,5.7,6.9,4.7,25.2,0.86,116,...,Female,White,Highschool,Lower-Middle,Never,Retired,0,0,0,1.0


In [72]:
df_test

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,triglycerides,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history
0,700000,45,4,100,4.3,6.8,6.2,25.5,0.84,123,...,111,Female,White,Highschool,Middle,Former,Employed,0,0,0
1,700001,35,1,87,3.5,4.6,9.0,28.6,0.88,120,...,145,Female,White,Highschool,Middle,Never,Unemployed,0,0,0
2,700002,45,1,61,7.6,6.8,7.0,28.5,0.94,112,...,184,Male,White,Highschool,Low,Never,Employed,0,0,0
3,700003,55,2,81,7.3,7.3,5.0,26.9,0.91,114,...,128,Male,White,Graduate,Middle,Former,Employed,0,0,0
4,700004,77,2,29,7.3,7.6,8.5,22.0,0.83,131,...,133,Male,White,Graduate,Low,Current,Unemployed,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,999995,59,3,185,6.3,7.3,4.4,22.8,0.81,108,...,126,Male,White,Highschool,Upper-Middle,Former,Employed,1,0,0
299996,999996,50,2,25,5.8,7.8,4.5,29.6,0.93,112,...,112,Male,Asian,Postgraduate,Lower-Middle,Never,Employed,0,0,0
299997,999997,63,1,252,5.2,7.5,8.5,25.1,0.77,129,...,135,Female,White,Highschool,Middle,Never,Employed,0,0,0
299998,999998,48,3,72,4.9,6.9,1.8,27.7,0.89,121,...,138,Male,White,Highschool,Low,Current,Retired,0,1,0


In [73]:
missing_values = df_train.isnull().sum()
missing_values = missing_values[missing_values > 0]

if not missing_values.empty:
    plt.figure(figsize=(10, 6))
    sns.barplot(x=missing_values.index, y=missing_values.values, palette='viridis')
    plt.xticks(rotation=90)
    plt.xlabel('Features')
    plt.ylabel('Missing Values')
    plt.title('Missing Values per Feature')
    plt.tight_layout()
    plt.show()
else:
    print("✅ No missing values found in the dataset.")

✅ No missing values found in the dataset.


In [74]:
y = df_train["diagnosed_diabetes"]
df_train["physical_inactivity"] = 1.0 / (df_train["physical_activity_minutes_per_week"] + 0.0000001)
df_test["physical_inactivity"] = 1.0 / (df_test["physical_activity_minutes_per_week"] + 0.0000001)
features = ["age",
    "alcohol_consumption_per_week",
    "physical_activity_minutes_per_week",
    "physical_inactivity",
    "diet_score",
    "sleep_hours_per_day",
    "screen_time_hours_per_day",
    "bmi",
    "waist_to_hip_ratio",
    "systolic_bp",
    "diastolic_bp",
    "heart_rate",
    "cholesterol_total",
    "hdl_cholesterol",
    "ldl_cholesterol",
    "triglycerides",
    "gender",
    "ethnicity",
    "education_level",
    "income_level",
    "smoking_status",
    "employment_status",
    "family_history_diabetes",
    "hypertension_history",
    "cardiovascular_history",]

X = pd.get_dummies(df_train[features], drop_first=True)
test = pd.get_dummies(df_test[features], drop_first=True)

# Align columns
X, test = X.align(test, join='inner', axis=1)
print(f"Train shape: {X.shape}, Test shape: {test.shape}")


Train shape: (700000, 37), Test shape: (300000, 37)


In [75]:
seeds = [i for i in range(5)]
def model_for_submission_lgbm(X_training, y, seeds):
    # train models and keep them in a list
    models = []
    kwargs = {
        # "objective": "binary",
        # "metric": "binary_logloss",
        # "boosting_type": "gbdt",
        "n_estimators": 2000,
        "learning_rate": 0.04,
        "num_leaves": 50,
        "max_depth": 6,
        "min_child_samples": 57,
        "subsample": 0.8,
        "colsample_bytree": 0.2,
        "reg_alpha": 9.5,
        "reg_lambda": 1.0e-08,
        "random_state": 44,
        "n_jobs": -1,
        "verbosity": -1,
    }
    for s in seeds:
        X_train, _, y_train, _ = train_test_split(X_training, y, test_size=0.2, random_state=s, stratify=y)
        X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=s, stratify=y_train)
        model = lightgbm.LGBMClassifier(**kwargs)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)])
        models.append(model)
    return models
models_lgb = model_for_submission_lgbm(X_training=X, y=df_train["diagnosed_diabetes"], seeds=seeds)
# --- train models and keep them ---

In [76]:
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import StratifiedKFold


def validate_model(X_training, y, n_classes=None, n_estimators=None, learning_rate=None, num_leaves=None, max_depth=None, min_child_samples=None, subsample=None, colsample_bytree=None, reg_alpha=None, reg_lambda=None):
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    oof_preds = np.zeros(len(y), dtype=float)

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X_training, y), start=1):
        X_tr, X_val = X_training.iloc[tr_idx], X_training.iloc[val_idx]
        y_tr, y_val = y[tr_idx], y[val_idx]

        kwargs = {
            "n_estimators": n_estimators,
            "learning_rate": learning_rate,
            "num_leaves": num_leaves,
            "max_depth": max_depth,
            "min_child_samples": min_child_samples,
            "subsample": subsample,
            "colsample_bytree": colsample_bytree,
            "reg_alpha": reg_alpha,
            "reg_lambda": reg_lambda,
            "random_state": 42 + fold,
            "n_jobs": -1,
        }
        m = lightgbm.LGBMClassifier(**kwargs)
        m.fit(X_tr, y_tr, eval_set=[(X_val, y_val)])
        oof_preds[val_idx] = m.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y, oof_preds)
    logloss = log_loss(y, oof_preds)
    brier = brier_score_loss(y, oof_preds)
    return auc, logloss, brier


In [77]:
for i, m in enumerate(models_lgb):
    print(i, type(m))


0 <class 'lightgbm.sklearn.LGBMClassifier'>
1 <class 'lightgbm.sklearn.LGBMClassifier'>
2 <class 'lightgbm.sklearn.LGBMClassifier'>
3 <class 'lightgbm.sklearn.LGBMClassifier'>
4 <class 'lightgbm.sklearn.LGBMClassifier'>


In [78]:
auc, logloss, brier = validate_model(
    X_training=X,
    y=df_train["diagnosed_diabetes"],
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8
)

print("AUC:", auc)
print("LogLoss:", logloss)
print("Brier Score:", brier)


AUC: 0.7253497210710081
LogLoss: 0.5840929056910444
Brier Score: 0.20062579350222876


In [79]:
X_test_final = test  # ← This is the one-hot encoded version you created earlier


probs_list = []

for model in models_lgb:
    p = model.predict_proba(X_test_final)[:, 1]  # probability of positive class
    probs_list.append(p)

# Average predictions across models
probs_mean = np.mean(probs_list, axis=0)

df_test["diagnosed_diabetes"] = probs_mean


# --------------------------------------------------
# 4. Build submission file
# --------------------------------------------------

submission = df_test[["id", "diagnosed_diabetes"]]
submission.to_csv("submission.csv", index=False)

print("Saved submission.csv with shape:", submission.shape)
print(submission.head())


Saved submission.csv with shape: (300000, 2)
       id  diagnosed_diabetes
0  700000            0.484217
1  700001            0.696360
2  700002            0.776628
3  700003            0.385895
4  700004            0.945904
